In [1]:
import feature_extractors_config as fe
import tensorflow as tf

import os
from collections import deque
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
import threading

In [2]:
dir_path = '../saved_experiments'
dir_list = list(os.walk(dir_path))[0][1]
model_names = dict(map(lambda x: (x, list(os.walk(dir_path+'/'+x))[0][1]), dir_list))

def print_city(model):
    print(os.path.join(dir_path, exp_w.value, model))
    return os.path.abspath(os.path.join(dir_path, exp_w.value, model))

def select_city(experiment):
    model_w.options = model_names[experiment]


exp_w = widgets.Dropdown(options=model_names.keys())
init = exp_w.value
model_w = widgets.Select(options=model_names[init])
j = widgets.interactive(print_city, model=model_w)
i = widgets.interactive(select_city, experiment=exp_w)
display(i)
display(j)

interactive(children=(Dropdown(description='experiment', options=('mobilenet_experiments_0', 'movenet_experime…

interactive(children=(Select(description='model', options=('0_gru', '1_lstm', 'logs'), value='0_gru'), Output(…

In [3]:
model = tf.saved_model.load(j.result)

# Set Hyperparameters

In [4]:
SEQ_LEN = 50
THRESHOLD = .6
FEAT_EXTRACTOR = fe.MovenetExtractor()

# Stop button
# ================

In [5]:
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the fa- prefix)
)
def view(button):
    cap = cv2.VideoCapture(0)
    out = cv2.VideoWriter('./TestVideos/LiveTesting/LSTM/SIGMOID/all3_low.avi', cv2.VideoWriter_fourcc(*'XVID'), 25.0, (int(cap.get(3)), int(cap.get(4))))

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    display_handle=display(None, display_id=True)
    i = 0
    frame_queue = deque(maxlen=SEQ_LEN)

    font                   = cv2.FONT_HERSHEY_SIMPLEX
    fontScale              = .5
    fontColor              = (0, 0,255)
    thickness              = 1
    lineType               = 2
    CLASS_LIST=['JumpingJack', 'PushUp', 'BodyWeightSquats']
    
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        height, width, channels = frame.shape

        frame_features = FEAT_EXTRACTOR.live_extract(frame)
        frame_queue.append(tf.keras.layers.Flatten()(frame_features))
        if len(frame_queue)<SEQ_LEN:
            if len(frame_queue)%10 == 0: print(len(frame_queue))
            continue
        x = tf.convert_to_tensor(list(frame_queue))
        x = tf.transpose(x, perm=[1, 0, 2])

#         for p in range(1):
#             for f in frame_features[0][p]:
#                 if f[2] > THRESHOLD:
#                     frame = cv2.circle(frame, (int(f[1]*width), int(f[0]*height)), radius=2, color=(0, (0 if p<1 else 255), 255), thickness=-1)
        
        if len(frame_queue) == SEQ_LEN:
            predicted_label_probabilities = model(x)[0]
            
            predicted_label = np.argmax(predicted_label_probabilities)
            predicted_class = CLASS_LIST[predicted_label]
        
        for idx, (pred, label) in enumerate(zip(predicted_label_probabilities.numpy(), ['Squats', 'Jumping Jacks', 'Pushups'])):
            cv2.putText(frame, label+': '+'{:f}'.format(pred) , (10, 30*(idx+1)), cv2.FONT_HERSHEY_SIMPLEX,1, (0, 0, 255), 2)

        out.write(frame)
        ret, frame = cv2.imencode('.jpeg', frame)

        display_handle.update(Image(data=frame.tobytes()))
        if stopButton.value:
            cap.release()
            out.release()
            display_handle.update(None)
            cv2.destroyAllWindows()
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Run
# ================

In [6]:
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

10
20
30
40


Exception in thread Thread-8:
Traceback (most recent call last):
  File "D:\Users\quinj\anaconda3\envs\Python Projects\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "D:\Users\quinj\anaconda3\envs\Python Projects\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\quinj\AppData\Local\Temp/ipykernel_15780/1261367598.py", line 48, in view
    predicted_label_probabilities = model(x)[0]
  File "D:\Users\quinj\anaconda3\envs\Python Projects\lib\site-packages\tensorflow\python\saved_model\load.py", line 744, in _call_attribute
    return instance.__call__(*args, **kwargs)
  File "D:\Users\quinj\anaconda3\envs\Python Projects\lib\site-packages\tensorflow\python\util\traceback_utils.py", line 153, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "D:\Users\quinj\anaconda3\envs\Python Projects\lib\site-packages\tensorflow\python\saved_model\function_deserialization.py", line 284, in restored_function